In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os
# Create a MirroredStrateg, If Multi-GPU available
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
tf.config.set_soft_device_placement(True)
# strategy = tf.distribute.MirroredStrategy(devices=['/gpu:1','/gpu:2']) 
# print('Number of GPUs being used: {}'.format(strategy.num_replicas_in_sync))
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from livelossplot.keras import PlotLossesCallback
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tqdm import tqdm
import efficientnet.keras as efn
import h5py, glob, re, cv2, math, matplotlib, pickle, gc
import tensorflow.keras as keras
import pandas as pd
from pandas import read_csv
import numpy as np
from scipy import stats
import itertools, random
from collections import Counter
# from cnn_utils import *
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from PIL import Image  
import pdb
from statistics import mode 
from IPython.display import clear_output

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

np.random.seed(1)
# Setting the seed for python random numbers
random.seed(1254)
# Setting the graph-level random seed.
tf.random.set_seed(89)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
!CUDA_VISIBLE_DEVICES=2

# strategy = tf.distribute.MirroredStrategy(devices=['/gpu:1','/gpu:2'])
# print('Number of GPUs being used: {}'.format(strategy.num_replicas_in_sync))
# print('Number of GPUs being used: {}'.format(strategy.num_replicas_in_sync))
# def setup_multi_node_training(): # IMPORTANT: SET UP TF_CONFIG FOR MULTINODE TRAINING HERE os.environ[“TF_FORCE_GPU_ALLOW_GROWTH”] = “true” tf.config.set_soft_device_placement(True) mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(tf.distribute.experimental.CollectiveCommunication.NCCL) # Constructs the configuration run_config = tf.estimator.RunConfig( train_distribute=mirrored_strategy, ) return run_config
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
num_class = 9
max_num_target = 3
print('num_class:', num_class)
print('num_target:', max_num_target)

num_class: 9
num_target: 3


## Data loading functions 

In [3]:
def load_pkl(filename):
    with open(filename, 'rb') as input:
        data = pickle.load(input)
    return data

In [4]:
def load_hdf5(filename):
    return h5py.File(filename, "r")

## Callbacks

In [5]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=20, restore_best_weights=True
)

In [6]:
def checkpoint_filepath(filename):
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = filename,
        save_weights_only = False,
        monitor = 'val_accuracy',
        mode = 'max',
        save_best_only = True)
    return checkpoint_callback

## Data generators

In [7]:
def data_generator(data, labels, batch_size=1):              
    """
    Yields the next training batch.
    data is an array  [[[frame1_filename,frame2_filename,…frame16_filename],label1], [[frame1_filename,frame2_filename,…frame16_filename],label2],……….].
    """
    num_samples = data.shape[0]
    
    while True:   
        for offset in range(0, num_samples, batch_size):
    #             print ('starting index: ', offset) 
            # Get the samples you'll use in this batch
            batch_samples = data[offset:offset+batch_size]
            label = labels[offset:offset+batch_size]
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            # For each example
            for i in range(0,batch_samples.shape[0]):
                X_train.append(batch_samples[i])
                y_train.append(label[i])

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            #X_train = np.rollaxis(X_train,1,4)
            y_train = np.array(y_train)

            # yield the next training batch            
            yield X_train, y_train

In [8]:
def data_generator_pred(data, labels, batch_size=1):              
    """
    Yields the next training batch.
    data is an array  [[[frame1_filename,frame2_filename,…frame16_filename],label1], [[frame1_filename,frame2_filename,…frame16_filename],label2],……….].
    """
    num_samples = data.shape[0]
    
    while True:   
        for offset in range(0, num_samples, batch_size):
    #             print ('starting index: ', offset) 
            # Get the samples you'll use in this batch
            batch_samples = data[offset:offset+batch_size]
            label = labels[offset:offset+batch_size]
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            # For each example
            for i in range(0,batch_samples.shape[0]):
                X_train.append(batch_samples[i])
                y_train.append(label[i])

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            #X_train = np.rollaxis(X_train,1,4)
            y_train = np.array(y_train)

            # yield the next training batch            
            yield X_train

## Multi-target network

In [9]:
def nn_singleact(input_shapex):
    inputlayer1 = Input(shape = input_shapex)
    x = Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(inputlayer1)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(120, activation='relu')(x)
    x = Dense(240, activation='relu')(x)
    x = Reshape(target_shape = (max_num_target, 240//max_num_target))(x)
#     x = Dropout(0.5)(x)
    out1 = TimeDistributed(Dense(num_class,activation='sigmoid'))(x)
#     x = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(x)
#     out1 = Dense(num_class,activation='softmax')(x)
#         out1 = TimeDistributed(Dense(num_class,activation='softmax'))(x, mask=maskinput)
#         model = keras.Model(inputs = [inputlayer1,maskinput], outputs = [out1])#, out2, out3])
    model = keras.Model(inputs = [inputlayer1], outputs = [out1])#, out2, out3])
    opt = Adam(lr=1e-3, decay=1e-3 / 200)
    model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics = ['accuracy'])
    return model 

In [10]:
# data

x = load_hdf5('datasets/single_act.hdf5')
x_train, x_test, y_train, y_test = [x['train_img'], x['test_img'], x['train_labels'], x['test_labels']]

y_temp = np.zeros((y_train.shape[0], max_num_target, num_class))
y_temp[:, 0, :] = y_train
y_train = y_temp

y_temp = np.zeros((y_test.shape[0], max_num_target, num_class))
y_temp[:, 0, :] = y_test
y_test = y_temp

print('Data shapes:')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Data shapes:
(321, 128, 128, 3)
(321, 3, 9)
(155, 128, 128, 3)
(155, 3, 9)


In [11]:
batch_size = 64
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
print('Input shape: ', input_shape)

Input shape:  (128, 128, 3)


# Model including multi-target samples

In [14]:
tr_per = list(range(0,101,10))
real_twoact_acc_ls = []
real_threeact_acc_ls = []
synt_twoact_acc_ls = []
synt_threeact_acc_ls = []
history_ls = []

# two act synt
x = load_hdf5('datasets/twoAct_orig.hdf5')
two_train_synt_x, two_test_synt_x, two_train_synt_y, two_test_synt_y = [x['train_img'], x['test_img'], x['train_labels'], x['test_labels']]
y_temp = np.zeros((two_train_synt_y.shape[0], max_num_target, num_class))
for i in range(len(two_train_synt_y)):
    y_temp[i, :two_train_synt_y.shape[1], :] = two_train_synt_y[i]
two_train_synt_y = y_temp
y_temp = np.zeros((two_test_synt_y.shape[0], max_num_target, num_class))
for i in range(len(two_test_synt_y)):
    y_temp[i, :two_test_synt_y.shape[1], :] = two_test_synt_y[i]
two_test_synt_y = y_temp

# three act synt
x = load_hdf5('datasets/threeAct_orig.hdf5')
three_train_synt_x, three_test_synt_x, three_train_synt_y, three_test_synt_y = [x['train_img'], x['test_img'], x['train_labels'], x['test_labels']]
y_temp = np.zeros((three_train_synt_y.shape[0], max_num_target, num_class))
for i in range(len(three_train_synt_y)):
    y_temp[i, :three_train_synt_y.shape[1], :] = three_train_synt_y[i]
three_train_synt_y = y_temp
y_temp = np.zeros((three_test_synt_y.shape[0], max_num_target, num_class))
for i in range(len(three_test_synt_y)):
    y_temp[i, :three_test_synt_y.shape[1], :] = three_test_synt_y[i]
three_test_synt_y = y_temp


# multi-target
x = load_pkl('datasets/multi_target.pkl')
x_train_multi, x_test_multi, y_train_multi, y_test_multi = [x[0], x[1], x[2], x[3]]
y_temp = np.zeros((y_train_multi.shape[0], max_num_target, num_class))
for i in range(len(y_train_multi)):
    y_temp[i, :len(y_train_multi[i]), :] = to_categorical(y_train_multi[i], num_class)
y_train_multi = y_temp
y_temp = np.zeros((y_test_multi.shape[0], max_num_target, num_class))
for i in range(len(y_test_multi)):
    y_temp[i, :len(y_test_multi[i]), :] = to_categorical(y_test_multi[i], num_class)
y_test_multi = y_temp
x_test_multi = np.concatenate([x_train_multi, x_test_multi], 0)
y_test_multi = np.concatenate([y_train_multi, y_test_multi], 0)

synt_acc_ls = []
real_acc_ls = []

for i in range(len(tr_per)):
    model_name = 'models/multi-target_trainPer_' + str(tr_per[i])
    print(str(i+1) + '/' + str(len(tr_per)) + ', ' + model_name)
    checkpoint_callback = checkpoint_filepath(model_name)
    nn = nn_singleact(input_shape)
    
    x_per_train = np.concatenate([x_train, two_train_synt_x[:int(len(two_train_synt_x)*tr_per[i]/100)],
                                three_train_synt_x[:int(len(three_train_synt_x)*tr_per[i]/100)]], axis=0)
    y_per_train = np.concatenate([y_train, two_train_synt_y[:int(len(two_train_synt_y)*tr_per[i]/100)],
                                three_train_synt_y[:int(len(three_train_synt_y)*tr_per[i]/100)]], axis=0)
    x_per_test = np.concatenate([x_test, two_test_synt_x, three_test_synt_x,
                               two_train_synt_x[int(len(two_train_synt_x)*tr_per[i]/100):],
                               three_train_synt_x[int(len(three_train_synt_x)*tr_per[i]/100):]], axis=0)
    y_per_test = np.concatenate([y_test, two_test_synt_y, three_test_synt_y,
                               two_train_synt_y[int(len(two_train_synt_y)*tr_per[i]/100):],
                               three_train_synt_y[int(len(three_train_synt_y)*tr_per[i]/100):]], axis=0)
    
    train_dataset = data_generator(x_per_train, y_per_train, batch_size)
    validation_dataset = data_generator(x_per_test, y_per_test, batch_size)
    
    history = nn.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=1000,
        shuffle = False,
        steps_per_epoch = math.ceil(len(y_per_train) / batch_size), validation_steps = math.ceil(len(y_per_test) / batch_size),
        callbacks=[checkpoint_callback],
        verbose = 1
    )
    history_ls.append(history)
    
    pred_dataset = data_generator_pred(x_per_test, y_per_test)
    preds = np.array([np.squeeze(nn.predict(next(pred_dataset))) for i in range(len(y_per_test))])
    
    correct = 0
    wrong = 0
    for i in range(len(y_per_test)):
        for j in range(len(y_per_test[i])):
            if sum(y_per_test[i][j]) != 0 or sum(preds[i][j]) > 0.5: # if there is target
                if np.argmax(y_per_test[i][j]) == np.argmax(preds[i][j]):
                    if y_per_test[i][j][0] == 1 and preds[i][j][0] < 0.5:
                        wrong += 1
                    else:
                        correct += 1
                else:
                    wrong += 1
    acc = correct / (correct + wrong) * 100
    synt_acc_ls.append(acc)
    print('Synt acc:', acc)
    
    pred_dataset = data_generator_pred(x_test_multi, y_test_multi)
    preds = np.array([np.squeeze(nn.predict(next(pred_dataset))) for i in range(len(y_test_multi))])
    
    correct = 0
    wrong = 0
    for i in range(len(y_test_multi)):
        for j in range(len(y_test_multi[i])):
            if sum(y_test_multi[i][j]) != 0 or sum(preds[i][j]) > 0.5: # if there is target
                if np.argmax(y_test_multi[i][j]) == np.argmax(preds[i][j]):
                    if y_test_multi[i][j][0] == 1 and preds[i][j][0] < 0.5:
                        wrong += 1
                    else:
                        correct += 1
                else:
                    wrong += 1
    acc = correct / (correct + wrong) * 100
    real_acc_ls.append(acc)
    print('Real acc:', acc)

1/11, models/multi-target_trainPer_0
Epoch 1/1000
6/6 [==============================] - 2s 269ms/step - loss: 0.4488 - accuracy: 0.0447 - val_loss: 0.3827 - val_accuracy: 0.0676
Epoch 2/1000
6/6 [==============================] - 2s 254ms/step - loss: 0.1503 - accuracy: 0.0218 - val_loss: 0.3889 - val_accuracy: 0.0737
Epoch 3/1000
6/6 [==============================] - 2s 254ms/step - loss: 0.1277 - accuracy: 0.0145 - val_loss: 0.4569 - val_accuracy: 0.0858
Epoch 4/1000
6/6 [==============================] - 2s 251ms/step - loss: 0.1043 - accuracy: 0.0696 - val_loss: 0.5365 - val_accuracy: 0.0898
Epoch 5/1000
6/6 [==============================] - 2s 254ms/step - loss: 0.1001 - accuracy: 0.1547 - val_loss: 0.6840 - val_accuracy: 0.1028
Epoch 6/1000
6/6 [==============================] - 2s 255ms/step - loss: 0.0846 - accuracy: 0.1817 - val_loss: 0.7854 - val_accuracy: 0.1204
Epoch 7/1000
6/6 [==============================] - 2s 331ms/step - loss: 0.0697 - accuracy: 0.2960 - val_loss:

Epoch 43/1000
6/6 [==============================] - 0s 36ms/step - loss: 1.6965e-05 - accuracy: 0.6386 - val_loss: 1.3320 - val_accuracy: 0.1656
Epoch 44/1000
6/6 [==============================] - 0s 36ms/step - loss: 1.5746e-05 - accuracy: 0.6324 - val_loss: 1.3355 - val_accuracy: 0.1653
Epoch 45/1000
6/6 [==============================] - 0s 36ms/step - loss: 1.4564e-05 - accuracy: 0.6345 - val_loss: 1.3375 - val_accuracy: 0.1653
Epoch 46/1000
6/6 [==============================] - 0s 36ms/step - loss: 1.3684e-05 - accuracy: 0.6324 - val_loss: 1.3414 - val_accuracy: 0.1659
Epoch 47/1000
6/6 [==============================] - 0s 37ms/step - loss: 1.2830e-05 - accuracy: 0.6324 - val_loss: 1.3441 - val_accuracy: 0.1656
Epoch 48/1000
6/6 [==============================] - 0s 36ms/step - loss: 1.2047e-05 - accuracy: 0.6324 - val_loss: 1.3450 - val_accuracy: 0.1659
Epoch 49/1000
6/6 [==============================] - 0s 36ms/step - loss: 1.1425e-05 - accuracy: 0.6334 - val_loss: 1.3469 -

6/6 [==============================] - 0s 36ms/step - loss: 2.2271e-06 - accuracy: 0.6293 - val_loss: 1.3800 - val_accuracy: 0.1647
Epoch 100/1000
6/6 [==============================] - 0s 37ms/step - loss: 2.1931e-06 - accuracy: 0.6293 - val_loss: 1.3804 - val_accuracy: 0.1647
Epoch 101/1000
6/6 [==============================] - 0s 36ms/step - loss: 2.1384e-06 - accuracy: 0.6293 - val_loss: 1.3803 - val_accuracy: 0.1647
Epoch 102/1000
6/6 [==============================] - 0s 36ms/step - loss: 2.0999e-06 - accuracy: 0.6293 - val_loss: 1.3812 - val_accuracy: 0.1647
Epoch 103/1000
6/6 [==============================] - 0s 36ms/step - loss: 2.0544e-06 - accuracy: 0.6293 - val_loss: 1.3816 - val_accuracy: 0.1647
Epoch 104/1000
6/6 [==============================] - 0s 37ms/step - loss: 2.0149e-06 - accuracy: 0.6303 - val_loss: 1.3816 - val_accuracy: 0.1647
Epoch 105/1000
6/6 [==============================] - 0s 37ms/step - loss: 1.9727e-06 - accuracy: 0.6303 - val_loss: 1.3816 - val_acc

Epoch 155/1000
6/6 [==============================] - 0s 36ms/step - loss: 8.7352e-07 - accuracy: 0.6355 - val_loss: 1.3954 - val_accuracy: 0.1647
Epoch 156/1000
6/6 [==============================] - 0s 36ms/step - loss: 8.6370e-07 - accuracy: 0.6355 - val_loss: 1.3954 - val_accuracy: 0.1647
Epoch 157/1000
6/6 [==============================] - 0s 37ms/step - loss: 8.5162e-07 - accuracy: 0.6355 - val_loss: 1.3954 - val_accuracy: 0.1647
Epoch 158/1000
6/6 [==============================] - 0s 36ms/step - loss: 8.4253e-07 - accuracy: 0.6355 - val_loss: 1.3954 - val_accuracy: 0.1647
Epoch 159/1000
6/6 [==============================] - 0s 35ms/step - loss: 8.3175e-07 - accuracy: 0.6366 - val_loss: 1.3954 - val_accuracy: 0.1644
Epoch 160/1000
6/6 [==============================] - 0s 37ms/step - loss: 8.2277e-07 - accuracy: 0.6366 - val_loss: 1.3954 - val_accuracy: 0.1644
Epoch 161/1000
6/6 [==============================] - 0s 36ms/step - loss: 8.1146e-07 - accuracy: 0.6366 - val_loss: 1

Epoch 211/1000
6/6 [==============================] - 0s 37ms/step - loss: 4.8140e-07 - accuracy: 0.6438 - val_loss: 1.4014 - val_accuracy: 0.1653
Epoch 212/1000
6/6 [==============================] - 0s 37ms/step - loss: 4.7710e-07 - accuracy: 0.6438 - val_loss: 1.4014 - val_accuracy: 0.1653
Epoch 213/1000
6/6 [==============================] - 0s 37ms/step - loss: 4.7232e-07 - accuracy: 0.6438 - val_loss: 1.4014 - val_accuracy: 0.1653
Epoch 214/1000
6/6 [==============================] - 0s 36ms/step - loss: 4.6809e-07 - accuracy: 0.6438 - val_loss: 1.4014 - val_accuracy: 0.1653
Epoch 215/1000
6/6 [==============================] - 0s 36ms/step - loss: 4.6401e-07 - accuracy: 0.6449 - val_loss: 1.4040 - val_accuracy: 0.1653
Epoch 216/1000
6/6 [==============================] - 0s 36ms/step - loss: 4.6008e-07 - accuracy: 0.6449 - val_loss: 1.4040 - val_accuracy: 0.1653
Epoch 217/1000
6/6 [==============================] - 0s 36ms/step - loss: 4.5558e-07 - accuracy: 0.6449 - val_loss: 1

...

Epoch 767/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2975 - val_loss: 1.2403 - val_accuracy: 0.4080
Epoch 768/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2975 - val_loss: 1.2403 - val_accuracy: 0.4080
Epoch 769/1000
15/15 [==============================] - 2s 107ms/step - loss: 0.1707 - accuracy: 0.2978 - val_loss: 1.2404 - val_accuracy: 0.4087
Epoch 770/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2986 - val_loss: 1.2403 - val_accuracy: 0.4080
Epoch 771/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.1706 - accuracy: 0.2986 - val_loss: 1.2403 - val_accuracy: 0.4087
Epoch 772/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.1707 - accuracy: 0.2993 - val_loss: 1.2404 - val_accuracy: 0.4087
Epoch 773/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.3000 - val_loss: 1.2404 - val_a

15/15 [==============================] - 0s 19ms/step - loss: 0.1728 - accuracy: 0.2580 - val_loss: 1.1391 - val_accuracy: 0.3760
Epoch 810/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1823 - accuracy: 0.2058 - val_loss: 1.2065 - val_accuracy: 0.1773
Epoch 811/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1998 - accuracy: 0.1812 - val_loss: 0.2319 - val_accuracy: 0.1327
Epoch 812/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.2023 - accuracy: 0.1449 - val_loss: 1.0689 - val_accuracy: 0.1713
Epoch 813/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1829 - accuracy: 0.1909 - val_loss: 0.8916 - val_accuracy: 0.1593
Epoch 814/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1777 - accuracy: 0.1775 - val_loss: 1.1166 - val_accuracy: 0.3640
Epoch 815/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.1748 - accuracy: 0.2283 - val_loss: 1.2217 - val_accuracy: 0.3633


15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2478 - val_loss: 1.2340 - val_accuracy: 0.3807
Epoch 866/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2478 - val_loss: 1.2341 - val_accuracy: 0.3807
Epoch 867/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2486 - val_loss: 1.2341 - val_accuracy: 0.3807
Epoch 868/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2489 - val_loss: 1.2342 - val_accuracy: 0.3813
Epoch 869/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2496 - val_loss: 1.2343 - val_accuracy: 0.3820
Epoch 870/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2496 - val_loss: 1.2344 - val_accuracy: 0.3827
Epoch 871/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2496 - val_loss: 1.2345 - val_accuracy: 0.3833


15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2667 - val_loss: 1.2374 - val_accuracy: 0.3993
Epoch 922/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2670 - val_loss: 1.2375 - val_accuracy: 0.3993
Epoch 923/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.1706 - accuracy: 0.2670 - val_loss: 1.2375 - val_accuracy: 0.4000
Epoch 924/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2681 - val_loss: 1.2375 - val_accuracy: 0.4013
Epoch 925/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2681 - val_loss: 1.2375 - val_accuracy: 0.4013
Epoch 926/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2681 - val_loss: 1.2376 - val_accuracy: 0.4013
Epoch 927/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2685 - val_loss: 1.2376 - val_accuracy: 0.4013


15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2822 - val_loss: 1.2383 - val_accuracy: 0.4033
Epoch 978/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2826 - val_loss: 1.2383 - val_accuracy: 0.4033
Epoch 979/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2826 - val_loss: 1.2384 - val_accuracy: 0.4027
Epoch 980/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2830 - val_loss: 1.2384 - val_accuracy: 0.4027
Epoch 981/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2826 - val_loss: 1.2383 - val_accuracy: 0.4020
Epoch 982/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2822 - val_loss: 1.2383 - val_accuracy: 0.4020
Epoch 983/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.1706 - accuracy: 0.2822 - val_loss: 1.2383 - val_accuracy: 0.4020


In [15]:
real_acc_ls

[12.5,
 18.75,
 29.166666666666668,
 25.0,
 25.0,
 20.833333333333336,
 41.66666666666667,
 27.083333333333332,
 25.0,
 39.58333333333333,
 27.083333333333332]

In [16]:
synt_acc_ls

[13.2857779759251,
 17.494089834515368,
 15.214105793450884,
 16.873315363881403,
 17.333333333333336,
 17.178683385579937,
 16.723549488054605,
 17.827715355805243,
 20.497925311203318,
 22.790697674418606,
 22.01058201058201]